In [1]:
import glob
import json

import numpy as np

In [2]:
seeds = [7, 11, 13]
models = ['cnn', 'mlp']
num_seeds = len(seeds)

## Table 1

In [3]:
score_types = ['top', 'avg']

algorithms = [
    'sup',
    'arora',
    'stochastic',    
    'deterministic',
    'pac-bayes'
]

In [4]:
for model in models:
    table_1_data = {}
    for algorithm in algorithms:
        results = {'top1': [], 'top5': [], 'mu5-top1': [], 'mu5-top5': [], 'avg2': [], 'mu5-avg2': []}    
        for score_type in score_types:
            for seed in seeds:
                fname = 'results/{}/{}-{}-{}.json'.format(model, algorithm, score_type, seed)
                data = json.load(open(fname))
                for k, v in data.items():
                    results[k].append(v)
        table_1_data[algorithm] = results


    top1_row = '& \\topone & '
    top5_row = '& \\topfive & '
    avg2_row = '& \\avgtwo & '

    score_format = '${:.1f}$'.format
    for k, data in table_1_data.items():

        top1_row += score_format(np.array(data['top1']).mean())
        top1_row += ' & '
        top1_row += score_format(np.array(data['mu5-top1']).mean())
        top1_row += ' && '

        top5_row += score_format(np.array(data['top5']).mean())
        top5_row += ' & '
        top5_row += score_format(np.array(data['mu5-top5']).mean())
        top5_row += ' && '

        avg2_row += score_format(np.array(data['avg2']).mean())
        avg2_row += ' & '
        avg2_row += score_format(np.array(data['mu5-avg2']).mean())
        avg2_row += ' && '
    print('\\\\ \n'.join([avg2_row[:-3], top1_row[:-3], top5_row[:-3]]) + '\\\\')
    if model == 'cnn':
        print('\\midrule\n\\multicolumn{2}{c}{\\texttt{AUSLAN}} \\\\')

& \avgtwo & $91.4$ & $87.6$ && $89.4$ & $85.5$ && $87.5$ & $83.4$ && $90.0$ & $87.1$ && $75.2$ & $70.9$ \\ 
& \topone & $25.3$ & $17.2$ && $22.5$ & $15.0$ && $16.9$ & $12.2$ && $21.4$ & $16.2$ && $6.8$ & $5.5$ \\ 
& \topfive & $57.8$ & $46.1$ && $52.9$ & $42.7$ && $46.2$ & $37.0$ && $54.0$ & $45.4$ && $23.0$ & $19.6$ \\
\midrule
\multicolumn{2}{c}{\texttt{AUSLAN}} \\
& \avgtwo & $79.1$ & $74.0$ && $76.1$ & $68.5$ && $76.2$ & $69.1$ && $76.4$ & $69.7$ && $74.6$ & $67.5$ \\ 
& \topone & $12.2$ & $7.5$ && $23.2$ & $11.4$ && $24.9$ & $11.6$ && $26.0$ & $12.4$ && $19.6$ & $9.9$ \\ 
& \topfive & $35.5$ & $24.8$ && $45.4$ & $26.0$ && $45.5$ & $26.9$ && $46.6$ & $27.8$ && $40.8$ & $24.3$ \\


## Table 2

In [5]:
def get_best_model_name(model_name_dir, criterion_key_in_json='lowest_val_loss'):

    if model_name_dir[-1] == '/':
        model_name_dir = model_name_dir[:-1]
    fnames = glob.glob('{}/*.json'.format(model_name_dir))

    lowest_val_loss = np.finfo(np.float(0.)).max

    best_model_fname = ''
    for fname in fnames:
        result = json.load(open(fname))
        if criterion_key_in_json not in result:
            print('{} field is not found in {}'.format(criterion_key_in_json, fname))
            continue

        if result[criterion_key_in_json] < lowest_val_loss:
            lowest_val_loss = result[criterion_key_in_json]
            best_model_fname = fname

    return best_model_fname.replace('json', 'pt')


In [6]:
def get_optimal(terms):
    lambda_array = np.arange(10, 10**7, 1)
    bounds = catoni_bound(lambda_array, terms)
    optimized_pb_bound = min(bounds)
    optimal_lambda = lambda_array[np.argmin(bounds)]
    return optimal_lambda, optimized_pb_bound

def catoni_bound(l, terms):
    M = terms['m']
    inner_exp = l / M  * terms['train-zero-one-loss'] + (terms['complexity'] + 2 * np.sqrt(M)) / M
    return (1. - np.exp(-inner_exp)) / (1. - np.exp(- l / M))        


In [7]:
float_format = ' ${:.3f}$ &'.format
lambda_format = ' $10^{}$ &'.format
kl_format = ' ${:,d}$ &'.format
optimal_lambda_format = ' ${:,}$ &'.format
line_break = '\\\\'

val_algorithms = [
    'stochastic',
    'deterministic',
]

results = []

for model in models:
    emp_risk_row = '& $\\Runhat(\\Qcal)$ & '
    test_risk_row = '& $\\Run(\\Qcal)$ &'
    emp_risk_f_row = '& $\\Runhat(\\fbf^*)$ & '
    test_risk_f_row = '& $\\Run(\\fbf^*)$ &'
    bound_row = '& Bound &'
    kl_row = '& $\\KL$ &'
    fixed_lambda_row = '& $\\lambda\\times m$ &'
    optimal_lambda_row = '& $\widehat{\lambda}\\times m$ &'
    
    for algorithm in val_algorithms:
        emp_risk = 0. 
        test_risk = 0.
        emp_risk_f = 0. 
        test_risk_f = 0.
        bound = 0.
        kl = 0.
        fixed_lambda = 0.
        sum_optimal_lambda = 0.
        for seed in seeds:
            fname = get_best_model_name(model_name_dir='weights/{}/{}/seed-{}'.format(model, algorithm, seed))
            data = json.load(open('./bounds/{}/pac-bayes-{}-{}.json'.format(model, seed, algorithm)))
            terms = data[fname]
            
            terms_f = json.load(open('./bounds/{}/pac-bayes-{}-{}-det.json'.format(model, seed, algorithm)))[fname]
            emp_risk_f += terms_f['train-zero-one-loss']
            test_risk_f += terms_f['test-zero-one-loss']
            del terms_f

            optimal_lambda, optimized_pb_bound = get_optimal(terms)
            
            print('\r', model, algorithm, seed, optimized_pb_bound, optimal_lambda, terms['lambda'], end='')

            emp_risk += terms['train-zero-one-loss']
            test_risk += terms['test-zero-one-loss']
            bound += optimized_pb_bound
            kl +=  terms['kl']
            fixed_lambda += terms['lambda']
            sum_optimal_lambda += optimal_lambda
            
        emp_risk_row += float_format(emp_risk / num_seeds)
        test_risk_row +=  float_format(test_risk / num_seeds)
        emp_risk_f_row += float_format(emp_risk_f / num_seeds)
        test_risk_f_row +=  float_format(test_risk_f / num_seeds)        
        bound_row += float_format(bound / num_seeds)
        kl_row += kl_format(int(kl / num_seeds))
        fixed_lambda_row +=  lambda_format(int(np.log10(fixed_lambda/num_seeds)))
        optimal_lambda_row += optimal_lambda_format(int(sum_optimal_lambda / num_seeds))
    
    # PAC-Bayes
    emp_risk = 0.
    test_risk = 0.
    emp_risk_f = 0.
    test_risk_f = 0.    
    bound = 0.
    kl = 0.
    fixed_lambda = 0.
    sum_optimal_lambda = 0.    
    for seed in seeds:
        data = json.load(open('./bounds/{}/pac-bayes-{}.json'.format(model, seed)))
        data_f = json.load(open('./bounds/{}/pac-bayes-{}-det.json'.format(model, seed)))
        
        best_l = 0
        best_bound = 10**7.
        best_terms = {}
        best_terms_f = {}
        for k, terms in data.items():
            optimal_lambda, optimized_pb_bound = get_optimal(terms)
            if optimized_pb_bound < best_bound:
                best_l = optimal_lambda
                best_terms = terms
                best_bound = optimized_pb_bound
                best_terms_f = data_f[k]               

        print('\r', model, 'PAC-Bayes', seed, best_bound, best_l, best_terms['lambda'], end='')

        emp_risk += best_terms['train-zero-one-loss']
        test_risk += best_terms['test-zero-one-loss']
        emp_risk_f += best_terms_f['train-zero-one-loss']
        test_risk_f += best_terms_f['test-zero-one-loss']
        bound += best_bound
        kl +=  int(best_terms['kl'])
        fixed_lambda += best_terms['lambda']
        sum_optimal_lambda += best_l
        
    print('\r', end='')

    emp_risk_row += float_format(emp_risk / num_seeds) + line_break
    test_risk_row += float_format(test_risk / num_seeds ) + line_break
    emp_risk_f_row += float_format(emp_risk_f / num_seeds) + line_break
    test_risk_f_row += float_format(test_risk_f / num_seeds ) + line_break    
    bound_row += float_format(bound / num_seeds) + line_break
    kl_row += kl_format( int(kl / num_seeds))  + line_break
    fixed_lambda_row += lambda_format(int(np.log10(fixed_lambda / num_seeds))) + line_break
    optimal_lambda_row += optimal_lambda_format(int(sum_optimal_lambda / num_seeds)) + line_break

    results.append('\n'.join([emp_risk_f_row, test_risk_f_row, emp_risk_row, test_risk_row, bound_row, kl_row, fixed_lambda_row, optimal_lambda_row]))

    if model == 'cnn':
        results.append('\n\\midrule\n\\multicolumn{2}{l}{\\texttt{AUSLAN}} \\\\\n')

In [8]:
print(''.join(results).replace(',', '\\,').replace('&\\\\', '\\\\'))

& $\Runhat(\fbf^*)$ &  $0.154$ & $0.131$ & $0.308$ \\
& $\Run(\fbf^*)$ & $0.189$ & $0.167$ & $0.315$ \\
& $\Runhat(\Qcal)$ &  $0.179$ & $0.170$ & $0.324$ \\
& $\Run(\Qcal)$ & $0.207$ & $0.197$ & $0.330$ \\
& Bound & $0.735$ & $0.721$ & $0.455$ \\
& $\KL$ & $31\,707$ & $30\,894$ & $1\,342$ \\
& $\lambda\times m$ & $10^5$ & $10^5$ & $10^4$ \\
& $\widehat{\lambda}\times m$ & $120\,915$ & $120\,470$ & $27\,954$ \\
\midrule
\multicolumn{2}{l}{\texttt{AUSLAN}} \\
& $\Runhat(\fbf^*)$ &  $0.001$ & $0.000$ & $0.032$ \\
& $\Run(\fbf^*)$ & $0.006$ & $0.005$ & $0.091$ \\
& $\Runhat(\Qcal)$ &  $0.001$ & $0.001$ & $0.005$ \\
& $\Run(\Qcal)$ & $0.007$ & $0.008$ & $0.020$ \\
& Bound & $0.069$ & $0.068$ & $0.034$ \\
& $\KL$ & $5\,064$ & $4\,996$ & $1\,369$ \\
& $\lambda\times m$ & $10^5$ & $10^5$ & $10^4$ \\
& $\widehat{\lambda}\times m$ & $366\,842$ & $368\,916$ & $196\,729$ \\
